In [118]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import pickle

from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, VotingRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

We have the necessary libraries needed, now let's load our dataset

In [64]:

playertrain_df = pd.read_csv("/content/male_players (legacy).csv") #for training
playertest_df =  pd.read_csv("/content/players_22 (1).csv") # for testing

<ipython-input-64-efbace2a5b5f>:2: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  playertrain_df = pd.read_csv("/content/male_players (legacy).csv") #for training
<ipython-input-64-efbace2a5b5f>:3: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  playertest_df =  pd.read_csv("/content/players_22 (1).csv") # for testing


In [65]:

playertrain_df.head()

,player_id,player_url,fifa_version,fifa_update,fifa_update_date,short_name,long_name,player_positions,overall,potential,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url
0,158023,/player/158023/lionel-messi/150002,15,2,2014-09-18,L. Messi,Lionel Andrés Messi Cuccittini,CF,93,95,...,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3,https://cdn.sofifa.net/players/158/023/15_120.png
1,20801,/player/20801/c-ronaldo-dos-santos-aveiro/150002,15,2,2014-09-18,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"LW, LM",92,92,...,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3,https://cdn.sofifa.net/players/020/801/15_120.png
2,9014,/player/9014/arjen-robben/150002,15,2,2014-09-18,A. Robben,Arjen Robben,"RM, LM, RW",90,90,...,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3,https://cdn.sofifa.net/players/009/014/15_120.png
3,41236,/player/41236/zlatan-ibrahimovic/150002,15,2,2014-09-18,Z. Ibrahimović,Zlatan Ibrahimović,ST,90,90,...,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3,17+3,https://cdn.sofifa.net/players/041/236/15_120.png
4,167495,/player/167495/manuel-neuer/150002,15,2,2014-09-18,M. Neuer,Manuel Peter Neuer,GK,90,90,...,40+3,40+3,36+3,36+3,38+3,38+3,38+3,36+3,87+3,https://cdn.sofifa.net/players/167/495/15_120.png


In [125]:
 playertest_df.head()

,sofifa_id,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,club_name,...,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,93,93.0,78000000.0,320000.0,34.0,170.0,72.0,73.0,Paris Saint-Germain,...,24.0,6.0,11.0,15.0,14.0,8.0,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,188545,92,92.0,119500000.0,270000.0,32.0,185.0,81.0,21.0,FC Bayern München,...,19.0,15.0,6.0,12.0,8.0,10.0,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png
2,20801,91,91.0,45000000.0,270000.0,36.0,187.0,83.0,11.0,Manchester United,...,24.0,7.0,11.0,15.0,14.0,11.0,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png
3,190871,91,91.0,129000000.0,270000.0,29.0,175.0,68.0,73.0,Paris Saint-Germain,...,29.0,9.0,9.0,15.0,15.0,11.0,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png
4,192985,91,91.0,125500000.0,350000.0,30.0,181.0,70.0,10.0,Manchester City,...,53.0,15.0,13.0,5.0,10.0,13.0,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png


In [67]:
playertrain_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Columns: 110 entries, player_id to player_face_url
dtypes: float64(18), int64(45), object(47)
memory usage: 135.6+ MB


In [68]:
playertest_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Columns: 110 entries, sofifa_id to nation_flag_url
dtypes: float64(16), int64(44), object(50)
memory usage: 16.1+ MB


In [69]:
print("Checking sum of missing value for Players 21(Train Data:)")
playertrain_df.isnull().sum()

Checking sum of missing value for Players 21(Train Data:)


player_id           0
player_url          0
fifa_version        0
fifa_update         0
fifa_update_date    0
                   ..
cb                  0
rcb                 0
rb                  0
gk                  0
player_face_url     0
Length: 110, dtype: int64

In [70]:
print("Checking sum of missing value for Players 22(Test Data:)")
playertest_df.isnull().sum()

Checking sum of missing value for Players 22(Test Data:)


sofifa_id               0
player_url              0
short_name              0
long_name               0
player_positions        0
                    ...  
player_face_url         0
club_logo_url          61
club_flag_url          61
nation_logo_url     18480
nation_flag_url         0
Length: 110, dtype: int64

In [73]:
total_rows_21 = playertrain_df.shape[0]
threshold_21 = int(0.3 * total_rows_21)

print("The threshold for Playertrain is", threshold_21)

The threshold for Playertrain is 48474


In [74]:
columns_to_drop = []
for column in playertrain_df.columns:
    if playertrain_df[column].isna().sum() > threshold_21:
        columns_to_drop.append(column)

Drop the columns:

In [75]:

playertrain_df = playertrain_df.drop(columns=columns_to_drop, axis=0)
playertest_df = playertest_df.drop(columns=columns_to_drop , axis=0)

Let's check info again:

In [76]:

playertrain_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Columns: 102 entries, player_id to player_face_url
dtypes: float64(14), int64(45), object(43)
memory usage: 125.7+ MB


In [77]:
playertest_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Columns: 102 entries, sofifa_id to nation_flag_url
dtypes: float64(12), int64(44), object(46)
memory usage: 15.0+ MB


In [84]:
drop_columns = ['player_url','long_name','dob','body_type','real_face','player_face_url']

playertest_df = playertest_df.drop(drop_columns, axis=1)
playertrain_df = playertrain_df.drop(drop_columns, axis=1)

KeyError: "['player_url', 'long_name', 'dob', 'body_type', 'real_face', 'player_face_url'] not found in axis"

In [85]:
#drop new identified columnas
drop_r_cols = ['short_name', 'player_positions', 'league_name', 'nationality_name', 'ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk']

playertrain_df = playertrain_df.drop(drop_r_cols, axis=1)
playertest_df = playertest_df.drop(drop_r_cols, axis=1)

KeyError: "['short_name', 'player_positions', 'league_name', 'nationality_name', 'ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk'] not found in axis"

In [86]:
## Filling missing numeric data with the mean value
num_imputer = SimpleImputer(strategy='mean')

## Filling missing categorical data with the most frequent value
cat_imputer = SimpleImputer(strategy='most_frequent')

In [87]:
# Selecting numerical and categorical features

num_features_train = playertrain_df.select_dtypes(include=[np.number]).columns.tolist()
cat_features_train = playertrain_df.select_dtypes(include=[object]).columns.tolist()

In [88]:
cat_features_train

['fifa_update_date',
 'club_name',
 'club_position',
 'club_joined_date',
 'preferred_foot',
 'work_rate']

In [115]:
# Removing the target variable from the features
num_features_train.remove('overall')  # 'overall' is the target variable

ValueError: list.remove(x): x not in list

In [91]:
#numeric imputation
playertrain_df[num_features_train] = num_imputer.fit_transform(playertrain_df[num_features_train])

#categorical imputation
playertrain_df[cat_features_train] = cat_imputer.fit_transform(playertrain_df[cat_features_train])

In [92]:
# Check if 'overall' is still in the list before attempting to remove it
if 'overall' in num_features_train:
    num_features_train.remove('overall')

# Exclude features not in player22_df BEFORE fitting
num_features_train = [col for col in num_features_train if col in playertest_df.columns]

#numeric imputation for training data
playertrain_df[num_features_train] = num_imputer.fit_transform(playertrain_df[num_features_train])

#categorical imputation for training data
# Ensure cat_features_train only contains features present in BOTH datasets
cat_features_train = [col for col in cat_features_train if col in playertest_df.columns]
playertrain_df[cat_features_train] = cat_imputer.fit_transform(playertrain_df[cat_features_train])

# Now, for player22_df, we use the same features as num_features_train
num_features_playertest = num_features_train
cat_features_playertest = [col for col in cat_imputer.feature_names_in_ if col in playertest_df.columns]

#numeric imputation for player22_df
playertest_df[num_features_playertest] = num_imputer.transform(playertest_df[num_features_playertest])

#categorical imputation for player22_df
playertest_df[cat_features_playertest] = cat_imputer.transform(playertest_df[cat_features_playertest])

In [93]:
playertrain_df.shape

(161583, 65)

In [94]:
playertest_df.shape

(19239, 65)

In [95]:
# Using `get_dummies` for one-hot encoding and dropping the first category
playertrain_encoded_df = pd.get_dummies(playertrain_df, columns=cat_features_train, drop_first=True)
playertest_encoded_df = pd.get_dummies(playertest_df, columns=cat_features_train, drop_first=True)

playertrain_encoded_df.head()  # display the first few rows to verify the changes

,player_id,fifa_version,fifa_update,fifa_update_date,overall,potential,value_eur,wage_eur,age,height_cm,...,club_position_SUB,preferred_foot_Right,work_rate_High/Low,work_rate_High/Medium,work_rate_Low/High,work_rate_Low/Low,work_rate_Low/Medium,work_rate_Medium/High,work_rate_Medium/Low,work_rate_Medium/Medium
0,158023.0,15.0,2.0,2014-09-18,93,95.0,100500000.0,550000.0,27.0,169.0,...,False,False,False,False,False,False,False,False,True,False
1,20801.0,15.0,2.0,2014-09-18,92,92.0,79000000.0,375000.0,29.0,185.0,...,False,True,True,False,False,False,False,False,False,False
2,9014.0,15.0,2.0,2014-09-18,90,90.0,54500000.0,275000.0,30.0,180.0,...,True,False,True,False,False,False,False,False,False,False
3,41236.0,15.0,2.0,2014-09-18,90,90.0,52500000.0,275000.0,32.0,195.0,...,False,True,False,False,False,False,False,False,True,False
4,167495.0,15.0,2.0,2014-09-18,90,90.0,63500000.0,300000.0,28.0,193.0,...,False,True,False,False,False,False,False,False,False,True


In [96]:
playertrain_encoded_df.shape

(161583, 1506)

In [112]:
playertrain_encoded_df.describe()

,player_id,fifa_version,fifa_update,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,...,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
count,161583.000000,161583.000000,161583.0,161583.000000,161583.000000,1.615830e+05,161583.000000,161583.000000,161583.000000,161583.000000,...,161583.000000,161583.000000,161583.000000,161583.000000,161583.000000,161583.00000,161583.000000,161583.000000,161583.000000,161583.000000
mean,214484.722353,19.125514,2.0,65.699071,70.744008,2.326770e+06,10855.409768,25.123181,181.240205,75.235031,...,48.668492,57.816892,45.757957,47.669996,45.698588,16.52961,16.274918,16.140374,16.288861,16.636973
std,34928.608856,2.559318,0.0,7.040855,6.259121,5.967471e+06,21821.763253,4.670207,6.750148,7.000456,...,15.652208,11.004799,20.453699,21.336404,20.935273,17.67047,16.834294,16.476466,16.998697,17.980143
min,2.000000,15.000000,2.0,40.000000,40.000000,1.000000e+03,500.000000,16.000000,154.000000,49.000000,...,5.000000,3.000000,1.000000,2.000000,3.000000,1.00000,1.000000,1.000000,1.000000,1.000000
25%,199159.000000,17.000000,2.0,61.000000,66.000000,3.250000e+05,2000.000000,21.000000,176.000000,70.000000,...,39.000000,53.000000,26.000000,27.000000,25.000000,8.00000,8.000000,8.000000,8.000000,8.000000
50%,220621.000000,19.000000,2.0,66.000000,70.000000,7.500000e+05,4000.000000,25.000000,181.000000,75.000000,...,50.000000,57.816892,50.000000,54.000000,52.000000,11.00000,11.000000,11.000000,11.000000,11.000000
75%,236958.000000,21.000000,2.0,70.000000,75.000000,1.900000e+06,10000.000000,28.000000,186.000000,80.000000,...,60.000000,65.000000,63.000000,66.000000,64.000000,14.00000,14.000000,14.000000,14.000000,14.000000
max,271817.000000,23.000000,2.0,94.000000,95.000000,1.940000e+08,575000.000000,54.000000,208.000000,110.000000,...,96.000000,96.000000,94.000000,94.000000,95.000000,91.00000,92.000000,95.000000,92.000000,94.000000


In [111]:
playertest_encoded_df.describe()

,sofifa_id,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,league_level,...,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
count,19239.000000,19239.000000,19239.000000,1.923900e+04,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,...,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000
mean,231468.086959,65.772182,71.079370,2.848438e+06,9023.815167,25.210822,181.299704,74.943032,50562.532520,1.354447,...,47.858724,57.929830,46.601746,48.045584,45.906700,16.406102,16.192474,16.055356,16.229274,16.491814
std,27039.717497,6.880232,6.086213,7.599112e+06,19439.558591,4.748235,6.863179,7.069434,54316.485284,0.746680,...,15.768583,12.159326,20.200807,21.232718,20.755683,17.574028,16.839528,16.564554,17.059779,17.884833
min,41.000000,47.000000,49.000000,9.000000e+03,500.000000,16.000000,155.000000,49.000000,1.000000,1.000000,...,7.000000,12.000000,4.000000,5.000000,5.000000,2.000000,2.000000,2.000000,2.000000,2.000000
25%,214413.500000,61.000000,67.000000,4.750000e+05,1000.000000,21.000000,176.000000,70.000000,479.000000,1.000000,...,38.000000,50.000000,29.000000,28.000000,25.000000,8.000000,8.000000,8.000000,8.000000,8.000000
50%,236543.000000,66.000000,71.000000,9.750000e+05,3000.000000,25.000000,181.000000,75.000000,1939.000000,1.000000,...,49.000000,59.000000,52.000000,56.000000,53.000000,11.000000,11.000000,11.000000,11.000000,11.000000
75%,253532.500000,70.000000,75.000000,2.100000e+06,8000.000000,29.000000,186.000000,80.000000,111138.000000,1.000000,...,60.000000,66.000000,63.000000,65.000000,63.000000,14.000000,14.000000,14.000000,14.000000,14.000000
max,264640.000000,93.000000,95.000000,1.940000e+08,350000.000000,54.000000,206.000000,110.000000,115820.000000,5.000000,...,93.000000,96.000000,93.000000,93.000000,92.000000,91.000000,92.000000,93.000000,92.000000,90.000000


In [99]:
# the target variable and features; drop non-numeric columns if necessary
X = playertrain_encoded_df.drop(columns=['overall'])
y = playertrain_encoded_df['overall']

In [113]:
# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=69)

In [124]:

from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()
model.fit(X_train, y_train)

importances = model.feature_importances_

indices = np.argsort(importances)[::-1][:10]


top_features = [X.columns[indices[i]] for i in range(10)]
print("\nTop 10 features with % Contribution:")

for i in range(10):
    print(f"{i + 1}.  {top_features[i]} ({round(importances[indices[i]]*100,2)}%)")


Top 10 features with % Contribution:
1.  value_eur (81.19%)
2.  age (7.67%)
3.  potential (4.56%)
4.  fifa_version (2.71%)
5.  movement_reactions (1.92%)
6.  player_id (0.83%)
7.  wage_eur (0.34%)
8.  defending (0.12%)
9.  international_reputation (0.07%)
10.  attacking_finishing (0.03%)


In [126]:

top_features = [X.columns[indices[i]] for i in range(10)]
print("\nTop 10 features with % Contribution:")

for i in range(10):
    print(f"{i + 1}.  {top_features[i]} ({round(importances[indices[i]]*100,2)}%)")



Top 10 features with % Contribution:
1.  value_eur (81.19%)
2.  age (7.67%)
3.  potential (4.56%)
4.  fifa_version (2.71%)
5.  movement_reactions (1.92%)
6.  player_id (0.83%)
7.  wage_eur (0.34%)
8.  defending (0.12%)
9.  international_reputation (0.07%)
10.  attacking_finishing (0.03%)


In [127]:
top_features = top_features[:5]

print('Features being used for model development are:\n')
top_features

Features being used for model development are:



['value_eur', 'age', 'potential', 'fifa_version', 'movement_reactions']

In [128]:
X_top_f = X[top_features]
X_top_f



,value_eur,age,potential,fifa_version,movement_reactions
0,100500000.0,27.0,95.0,15.0,94.0
1,79000000.0,29.0,92.0,15.0,90.0
2,54500000.0,30.0,90.0,15.0,89.0
3,52500000.0,32.0,90.0,15.0,85.0
4,63500000.0,28.0,90.0,15.0,89.0
...,...,...,...,...,...
161578,110000.0,18.0,61.0,23.0,39.0
161579,110000.0,19.0,58.0,23.0,42.0
161580,110000.0,19.0,58.0,23.0,50.0
161581,150000.0,17.0,70.0,23.0,45.0


Now let's scale our features which is our independent variables

In [129]:
# Initialize the scaler
scaler = StandardScaler()

# Scale the features
X_scaled = scaler.fit_transform(X_top_f)

# The features are now scaled and ready for training the model.
X_scaled_df = pd.DataFrame(X_scaled, columns=X_top_f.columns)

X_scaled_df.head()

,value_eur,age,potential,fifa_version,movement_reactions
0,16.451449,0.401872,3.875315,-1.611964,3.535775
1,12.848571,0.830120,3.396013,-1.611964,3.099571
2,8.742966,1.044244,3.076478,-1.611964,2.990520
3,8.407815,1.472491,3.076478,-1.611964,2.554317
4,10.251147,0.615996,3.076478,-1.611964,2.990520


In [130]:
#Saving scaler to use in deployment

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)


In [131]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=84)

In [132]:
def train_model(model, param_grid, X, y):
    '''
        Trains a model using grid search with cross-validation and returns the best model.
        Parameters:
            model: scikit-learn model
            param_grid: dictionary with parameters to try
            X: features(independent variables)
            y: target(dependent variable)
    '''
    cv = KFold(n_splits=7 , random_state=69, shuffle=True)

    # Grid search with cross-validation
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=cv, scoring='neg_mean_absolute_error', n_jobs=-1)
    grid_search.fit(X, y)

    # Results of the grid search
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Best score (MAE): {-grid_search.best_score_}")  # We negate the score because grid search maximizes performance (so it negates the scores)

    return grid_search.best_estimator_  # Returns the best model

## Model 1: XGBoost

In [ ]:
print("\nTraining XGBoost...")
xgb_model = xgb.XGBRegressor(random_state=42)
xgb_params = {
    'n_estimators': [100, 500, 1000],
    'learning_rate': [0.1, 0.001, 0.01],
    'max_depth': [3, 5, 9, 15],
    'colsample_bytree': [0.5, 0.75, 1]
}
best_xgb = train_model(xgb_model, xgb_params, X_train, y_train)


Training XGBoost...


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


## Model 2: Gradient Bossting Regressor

In [ ]:
print("\nTraining Gradient Boosting...")
gbr_model = GradientBoostingRegressor(random_state=63)
gbr_params = {
    'n_estimators': [100, 500, 1000],
    'learning_rate': [0.1, 0.001, 0.01],
    'max_depth': [9, 15]
}
best_gbr = train_model(gbr_model, gbr_params, X_train, y_train)


Training Gradient Boosting...
Best parameters: {'learning_rate': 0.1, 'max_depth': 15, 'n_estimators': 500}
Best score (MAE): 0.28019891850304995


In [ ]:
print("\nTraining Random Forest...")
rf_model = RandomForestRegressor(random_state=39)
rf_params = {
    'n_estimators': [500,1000],
    'max_depth': [12, 15],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}
best_rf = train_model(rf_model, rf_params, X_train, y_train)


Training Random Forest...
Best parameters: {'max_depth': 15, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 1000}
Best score (MAE): 0.2748741579368859


In [ ]:
# Create an ensemble model
ensemble = VotingRegressor(
    estimators=[
        ('xgb', best_xgb),
        ('gbr', best_gbr),
        ('rf', best_rf)
    ]
)

In [ ]:
# Fit model on the training data
print("\nTraining Ensemble Model...")
ensemble.fit(X_train, y_train)

# Predict and evaluate on the training set
train_pred = ensemble.predict(X_train)
train_mae = mean_absolute_error(y_train, train_pred)
print(f"Ensemble model MAE on training set: {train_mae}")


Training Ensemble Model...
Ensemble model MAE on training set: 0.04907840399452329


Now we have our trained Models. We are moving on to evaluations on the test set to see how they perform. Before Let's save so we don't have to incur cost of training if runtime fails

# Saving Models

In [ ]:
%cd "/content/drive/MyDrive/Cap_Mid"

/content/drive/MyDrive/Cap_Mid


In [ ]:
with open('best_xgb_model.pkl', 'wb') as file:
    pickle.dump(best_xgb, file)

with open('best_gbr_model.pkl', 'wb') as file:
    pickle.dump(best_gbr, file)

with open('best_rf_model.pkl', 'wb') as file:
    pickle.dump(best_rf, file)

with open('ensemble_model.pkl', 'wb') as file:
    pickle.dump(ensemble, file)

Test if model saved well

In [ ]:
with open('ensemble_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

predictions = loaded_model.predict(X_test)

en_mae = mean_absolute_error(y_test, predictions)

print(f"Ensemble model MAE on test set: {en_mae}")

Ensemble model MAE on test set: 0.2637278347406203


Saved Well

# Evaluation

We are going to do two evaluations, one on the test set seperated from the training data. The other on `Players 22` an unseen dataset similar to the data used to train models

## Test Set Evaluations

In [ ]:
print("\nEvaluating XGBoost...")

#predict on test set
pred_xgb = best_xgb.predict(X_test)
xgb_mae = mean_absolute_error(y_test, pred_xgb)

print(f"XGBoost model MAE on test set: {xgb_mae:.2f}")


Evaluating XGBoost...
XGBoost model MAE on test set: 0.28


In [ ]:
print("\nEvaluating Gradient Boost...")

#predict on test set
pred_gbr = best_gbr.predict(X_test)
gbr_mae = mean_absolute_error(y_test, pred_gbr)

print(f"Gradient Boost Regressor model MAE on test set: {gbr_mae}:.2f")


Evaluating Gradient Boost...
Gradient Boost Regressor model MAE on test set: 0.27734439088845086:.2f


In [ ]:
print("\nEvaluating Random Forest...")

#predict on test set
pred_rf = best_rf.predict(X_test)
rf_mae = mean_absolute_error(y_test, pred_rf)

print(f"Random Forest Regressor model MAE on test set: {rf_mae:.2f}")


Evaluating Random Forest...
Random Forest Regressor model MAE on test set: 0.27


In [ ]:
print("\nEvaluating Ensemble...")

#predict on test set
pred_en = ensemble.predict(X_test)
en_mae = mean_absolute_error(y_test, pred_en)

print(f"Ensemble model MAE on test set: {en_mae:.2f}")


Evaluating Ensemble...
Ensemble model MAE on test set: 0.26


## Player 22 Evaluations


Here we will test our trained models further on `player22` data, the data has been preprocessed already. We only have to extract just the top features needed

In [ ]:
player22_encoded_df['overall']

0        93
1        92
2        91
3        91
4        91
         ..
19234    47
19235    47
19236    47
19237    47
19238    47
Name: overall, Length: 19239, dtype: int64

In [ ]:
top_features

['value_eur', 'release_clause_eur', 'age', 'potential', 'movement_reactions']

In [ ]:
player22_encoded_df[top_features]

,value_eur,release_clause_eur,age,potential,movement_reactions
0,78000000.0,144300000.0,34.0,93.0,94.0
1,119500000.0,197200000.0,32.0,92.0,93.0
2,45000000.0,83300000.0,36.0,91.0,94.0
3,129000000.0,238700000.0,29.0,91.0,89.0
4,125500000.0,232200000.0,30.0,91.0,91.0
...,...,...,...,...,...
19234,70000.0,114000.0,22.0,52.0,53.0
19235,110000.0,193000.0,19.0,59.0,49.0
19236,100000.0,175000.0,21.0,55.0,46.0
19237,110000.0,239000.0,19.0,60.0,48.0


In [ ]:
#Get player 22 info
y_22 = player22_encoded_df['overall']
X_22 = player22_encoded_df[top_features]


In [ ]:
#Scale input

X_scaled_22 = scaler.fit_transform(X_22)

# The features are now scaled and ready for training the model.
X22_scaled_df = pd.DataFrame(X_scaled_22, columns=X_22.columns)

In [ ]:
#reassign
X_22 = X22_scaled_df

X_22.head()

,value_eur,release_clause_eur,age,potential,movement_reactions
0,9.889601,9.591752,1.851089,3.60178,3.599846
1,15.350958,13.244084,1.429869,3.43747,3.489252
2,5.546836,5.380179,2.272309,3.27316,3.599846
3,16.601147,16.109335,0.798039,3.27316,3.046874
4,16.140551,15.660560,1.008649,3.27316,3.268063


Using saved models here.

### Loading Saved Models

In [ ]:
#move to directory where models are saved
%cd "/content/drive/MyDrive/Cap_Mid"

/content/drive/MyDrive/Cap_Mid


In [ ]:
with open('best_xgb_model.pkl', 'rb') as file:
    lbest_xgb = pickle.load(file)

with open('best_gbr_model.pkl', 'rb') as file:
    lbest_gbr = pickle.load(file)

with open('best_rf_model.pkl', 'rb') as file:
    lbest_rf = pickle.load(file)

with open('ensemble_model.pkl', 'rb') as file:
    lensemble = pickle.load(file)

### Testing

In [ ]:
print("\nEvaluating XGBoost...")

#predict on test set
pred_xgb = lbest_xgb.predict(X_22)
xgb_mae = mean_absolute_error(y_22, pred_xgb)

print(f"XGBoost model MAE on Players 22 set: {xgb_mae:.2f}")


Evaluating XGBoost...
XGBoost model MAE on Players 22 set: 0.68


In [ ]:
print("\nEvaluating Random Forest...")

#predict on test set
pred_rf = lbest_rf.predict(X_22)
rf_mae = mean_absolute_error(y_22, pred_rf)

print(f"Random Forest Regressor model MAE on Players 22 set: {rf_mae:.2f}")


Evaluating Random Forest...
Random Forest Regressor model MAE on Players 22 set: 0.58


In [ ]:
print("\nEvaluating Gradient Boost...")

#predict on test set
pred_gbr = lbest_gbr.predict(X_22)
gbr_mae = mean_absolute_error(y_22, pred_gbr)

print(f"Gradient Boost Regressor model MAE on Players 22 set: {gbr_mae:.2f}")


Evaluating Gradient Boost...
Gradient Boost Regressor model MAE on Players 22 set: 0.62


In [ ]:
print("\nEvaluating Ensemble...")

#predict on test set
pred_en = lensemble.predict(X_22)
en_mae = mean_absolute_error(y_22, pred_en)

print(f"Ensemble model MAE on Players 22 set: {en_mae:.2f}")



Evaluating Ensemble...
Ensemble model MAE on Players 22 set: 0.59


In [ ]:
!pip freeze > requirements.txt